In [1]:
# packages
import pandas as pd
import numpy as np
import sqlite3

# Load Excel file

In [2]:
# Load excel file, Sheet1
df = pd.read_excel('Data_with_Coordinates.xlsx', sheet_name='Sheet1') #, index_col=0)

print(df.shape)

df.head(2) # displaying the two first rows

(1092, 9)


,Unnamed: 0,Fournisseurs,n° facture,Montant,Date de facture,Date d'échéance,n° sem,Mis en paie.,E-MAIL
0,0,Capsule Corp,115968610,74.0,2021-03-29,2021-04-12,15,2021-12-26 19:33:52,contact@capsule-corp.fr
1,1,Cyberdyne,21040827,1008.0,2021-04-09,2021-04-12,15,2021-12-26 19:34:04,contact@cyberdyne.fr


In [3]:
df['n° facture'].nunique() # we can consider n° facture as primary key when using SQL tables, 
                            #since each row corresponds to an unique bill

1092

#  id as a primary key 

In [4]:
# rename the first column, to set id as the primary key.

df.rename(columns = {'Unnamed: 0':'id'}, inplace = True) #id refers to bills

df["id"] = df["id"].apply(lambda x:x+1) # starting from 1
df.head()

,id,Fournisseurs,n° facture,Montant,Date de facture,Date d'échéance,n° sem,Mis en paie.,E-MAIL
0,1,Capsule Corp,115968610,74.00,2021-03-29,2021-04-12,15,2021-12-26 19:33:52,contact@capsule-corp.fr
1,2,Cyberdyne,21040827,1008.00,2021-04-09,2021-04-12,15,2021-12-26 19:34:04,contact@cyberdyne.fr
2,3,Geugène Industrie,2103006,2842.94,2021-03-15,2021-04-14,15,2021-12-26 19:34:05,contact@geugene.fr
3,4,Luthor Corp,F1920074,1325.00,2021-01-15,2021-04-15,15,2021-12-26 19:34:15,contact@luther.fr
4,5,Luthor Corp,F1920110,444.00,2021-01-15,2021-04-15,15,2021-12-26 19:34:16,contact@luther.fr


# Adding id_fournisseurs key

In [5]:
#Multiple rows are with the same provider
# We make a new column to represent providers by their id.
#Using for loop,dictionary and map functions 

def map_index(liste=[], start_id=1, end_id=11): #end_id=len(liste)+1
    list2 = list(np.arange(start_id, end_id))
    
    dic = {}
    for i,j in enumerate(liste):
        dic[j] = i+1
    return dic

L = list(df['Fournisseurs'].unique())

res = map_index(liste = L, start_id = 1, end_id=len(L)+1)
res.items()

dict_items([('Capsule Corp', 1), ('Cyberdyne', 2), ('Geugène Industrie', 3), ('Luthor Corp', 4), ('Drake & Sweeney', 5), ('Slusho', 6), ('Pyramid Transnational', 7), ('Sienar Technologies', 8), ('Beaumont-Liégard', 9), ('Cordell SA', 10), ('Tech Con', 11), ('Factory Mode', 12), ('Altra Automotive', 13), ('Umbrella Corporation', 14), ('LRCorp', 15), ('Zorg\xa0Corporation', 16), ('Le\xa0Yankee', 17), ('Wernham Hogg Paper Company', 18), ('Lacuna Inc', 19), ('Vladis Entreprise', 20), ('Sofroco-Gedec', 21), ('Bross & Clackwell', 22), ('World Company', 23), ('Groupe W.', 24), ('Cross Technological Enterprises\xa0', 25), ('Wayne Enterprises', 26), ('Rosen Corporation', 27), ('Aaltra', 28), ('Beau-Line', 29), ('General Broadcasting Corporation', 30), ('Tricatel', 31), ('Big Jerry Cab Co', 32), ('Soylent Green', 33), ('Protovision', 34), ('LexCorp', 35), ('Blue Star Airlines', 36), ('Dapper Dan', 37), ('Dinoco', 38), ('Nelson & Murdock', 39), ('Wascomed', 40), ('Yamagato Industries', 41), ('Eur

In [6]:
df["id_fournisseurs"] = df["Fournisseurs"].map(res)

df.head(7)

,id,Fournisseurs,n° facture,Montant,Date de facture,Date d'échéance,n° sem,Mis en paie.,E-MAIL,id_fournisseurs
0,1,Capsule Corp,115968610,74.00,2021-03-29,2021-04-12,15,2021-12-26 19:33:52,contact@capsule-corp.fr,1
1,2,Cyberdyne,21040827,1008.00,2021-04-09,2021-04-12,15,2021-12-26 19:34:04,contact@cyberdyne.fr,2
2,3,Geugène Industrie,2103006,2842.94,2021-03-15,2021-04-14,15,2021-12-26 19:34:05,contact@geugene.fr,3
3,4,Luthor Corp,F1920074,1325.00,2021-01-15,2021-04-15,15,2021-12-26 19:34:15,contact@luther.fr,4
4,5,Luthor Corp,F1920110,444.00,2021-01-15,2021-04-15,15,2021-12-26 19:34:16,contact@luther.fr,4
5,6,Luthor Corp,F1920112,757.00,2021-01-15,2021-04-15,15,2021-12-26 19:34:17,contact@luther.fr,4
6,7,Luthor Corp,F1920113,925.00,2021-01-15,2021-04-15,15,2021-12-26 19:34:17,contact@luther.fr,4


# Rename column labels 

In [7]:
#to store data into a database, we need for instance to modify columns labels in .db file
df_sql = df.copy() # make a copy 

In [8]:
df_sql.columns

Index(['id', 'Fournisseurs', 'n° facture', 'Montant', 'Date de facture',
       'Date d'échéance', 'n° sem', 'Mis en paie.', 'E-MAIL',
       'id_fournisseurs'],
      dtype='object')

In [9]:
col = []
for i in df_sql.columns:
    i = i.replace('° ','_').replace("d'", "").replace(' ', '_').replace("-", "").replace("é", "e").replace(".","")
    print(i)
    col.append(i)
df_sql.columns = col

id
Fournisseurs
n_facture
Montant
Date_de_facture
Date_echeance
n_sem
Mis_en_paie
EMAIL
id_fournisseurs


In [10]:
df_sql.columns

Index(['id', 'Fournisseurs', 'n_facture', 'Montant', 'Date_de_facture',
       'Date_echeance', 'n_sem', 'Mis_en_paie', 'EMAIL', 'id_fournisseurs'],
      dtype='object')

# Set index

In [11]:
df_sql.head(7)

,id,Fournisseurs,n_facture,Montant,Date_de_facture,Date_echeance,n_sem,Mis_en_paie,EMAIL,id_fournisseurs
0,1,Capsule Corp,115968610,74.00,2021-03-29,2021-04-12,15,2021-12-26 19:33:52,contact@capsule-corp.fr,1
1,2,Cyberdyne,21040827,1008.00,2021-04-09,2021-04-12,15,2021-12-26 19:34:04,contact@cyberdyne.fr,2
2,3,Geugène Industrie,2103006,2842.94,2021-03-15,2021-04-14,15,2021-12-26 19:34:05,contact@geugene.fr,3
3,4,Luthor Corp,F1920074,1325.00,2021-01-15,2021-04-15,15,2021-12-26 19:34:15,contact@luther.fr,4
4,5,Luthor Corp,F1920110,444.00,2021-01-15,2021-04-15,15,2021-12-26 19:34:16,contact@luther.fr,4
5,6,Luthor Corp,F1920112,757.00,2021-01-15,2021-04-15,15,2021-12-26 19:34:17,contact@luther.fr,4
6,7,Luthor Corp,F1920113,925.00,2021-01-15,2021-04-15,15,2021-12-26 19:34:17,contact@luther.fr,4


In [12]:
# setting first name as index column
df_sql.set_index('id', inplace=True)

In [13]:
df_sql.head(7)

,Fournisseurs,n_facture,Montant,Date_de_facture,Date_echeance,n_sem,Mis_en_paie,EMAIL,id_fournisseurs
id,,,,,,,,,
1,Capsule Corp,115968610,74.00,2021-03-29,2021-04-12,15,2021-12-26 19:33:52,contact@capsule-corp.fr,1
2,Cyberdyne,21040827,1008.00,2021-04-09,2021-04-12,15,2021-12-26 19:34:04,contact@cyberdyne.fr,2
3,Geugène Industrie,2103006,2842.94,2021-03-15,2021-04-14,15,2021-12-26 19:34:05,contact@geugene.fr,3
4,Luthor Corp,F1920074,1325.00,2021-01-15,2021-04-15,15,2021-12-26 19:34:15,contact@luther.fr,4
5,Luthor Corp,F1920110,444.00,2021-01-15,2021-04-15,15,2021-12-26 19:34:16,contact@luther.fr,4
6,Luthor Corp,F1920112,757.00,2021-01-15,2021-04-15,15,2021-12-26 19:34:17,contact@luther.fr,4
7,Luthor Corp,F1920113,925.00,2021-01-15,2021-04-15,15,2021-12-26 19:34:17,contact@luther.fr,4


In [14]:
df_sql.to_csv('Data_with_Coordinates.csv', index=False)

# SQL queries

In [15]:
#pip install ipython-sql

In [48]:
cnn = sqlite3.connect('data_fournisseurs.db') #db file 0Ko

In [49]:
df_sql.to_sql('factures', cnn) # load dataframe 'df_sql' into 'factures' table

In [18]:
%load_ext sql

In [19]:
%sql sqlite:///data_fournisseurs.db

## Query 1 : Sum amounts by week number 

In [20]:
%%sql

SELECT * FROM factures LIMIT 4

 * sqlite:///data_fournisseurs.db
Done.


id,Fournisseurs,n_facture,Montant,Date_de_facture,Date_echeance,n_sem,Mis_en_paie,EMAIL,id_fournisseurs
1,Capsule Corp,115968610,74.0,2021-03-29 00:00:00,2021-04-12 00:00:00,15,2021-12-26 19:33:52,contact@capsule-corp.fr,1
2,Cyberdyne,21040827,1008.0,2021-04-09 00:00:00,2021-04-12 00:00:00,15,2021-12-26 19:34:04,contact@cyberdyne.fr,2
3,Geugène Industrie,2103006,2842.94,2021-03-15 00:00:00,2021-04-14 00:00:00,15,2021-12-26 19:34:05,contact@geugene.fr,3
4,Luthor Corp,F1920074,1325.0,2021-01-15 00:00:00,2021-04-15 00:00:00,15,2021-12-26 19:34:15,contact@luther.fr,4


In [21]:
%%sql
SELECT Fournisseurs, Montant, n_facture
FROM factures LIMIT 4

 * sqlite:///data_fournisseurs.db
Done.


Fournisseurs,Montant,n_facture
Capsule Corp,74.0,115968610
Cyberdyne,1008.0,21040827
Geugène Industrie,2842.94,2103006
Luthor Corp,1325.0,F1920074


In [22]:
%%sql 

SELECT n_sem AS Semaine, cast(ROUND(SUM(Montant),0) as int) AS Montants, 

COUNT(n_facture) AS Nbre_Factures

FROM factures WHERE n_sem BETWEEN 8 AND 22 GROUP BY n_sem

 * sqlite:///data_fournisseurs.db
Done.


Semaine,Montants,Nbre_Factures
8,543614,15
9,676898,13
10,214533,16
15,346030,21
16,348875,14
17,732764,27
18,350854,19
19,299996,10
20,626353,34
21,418612,36


## Query 2 : Count bills number by provider

In [24]:
%%sql 

SELECT Fournisseurs,
COUNT(n_facture) AS Nbre_Factures

FROM factures WHERE n_sem BETWEEN 8 AND 22 GROUP BY id_fournisseurs

 * sqlite:///data_fournisseurs.db
Done.


Fournisseurs,Nbre_Factures
Capsule Corp,1
Cyberdyne,1
Geugène Industrie,5
Luthor Corp,163
Drake & Sweeney,6
Slusho,1
Pyramid Transnational,3
Sienar Technologies,16
Beaumont-Liégard,1
Cordell SA,6


## Query 3 : Count bills number by provider, in descending order

In [47]:
%%sql 

SELECT Fournisseurs,
COUNT(n_facture) AS Nbre_Factures

FROM factures WHERE n_sem BETWEEN 8 AND 22 GROUP BY id_fournisseurs ORDER BY Nbre_Factures DESC

 * sqlite:///data_fournisseurs.db
Done.


Fournisseurs,Nbre_Factures
Luthor Corp,163
Sienar Technologies,16
Drake & Sweeney,6
Cordell SA,6
Geugène Industrie,5
World Company,5
Pyramid Transnational,3
Bross & Clackwell,3
Altra Automotive,2
Zorg Corporation,2


In [25]:
%%sql 

SELECT Fournisseurs,
COUNT(n_facture) AS Nbre_Factures

FROM factures GROUP BY id_fournisseurs ORDER BY Nbre_Factures DESC

 * sqlite:///data_fournisseurs.db
Done.


Fournisseurs,Nbre_Factures
Luthor Corp,798
Sienar Technologies,69
Vladis Entreprise,29
Drake & Sweeney,20
Cordell SA,17
Geugène Industrie,15
World Company,11
Capsule Corp,10
Altra Automotive,10
Wernham Hogg Paper Company,8


In [26]:
# Check out the result using pandas dataframe  

In [27]:
df_sql['id_fournisseurs'].value_counts()

4     798
8      69
20     29
5      20
10     17
3      15
23     11
1      10
13     10
18      8
32      8
21      8
27      7
22      7
25      6
24      6
7       6
39      5
9       4
30      4
17      3
37      3
36      3
28      3
31      2
44      2
41      2
35      2
34      2
16      2
6       2
12      2
14      2
19      2
29      1
33      1
26      1
2       1
11      1
38      1
40      1
15      1
42      1
43      1
45      1
46      1
Name: id_fournisseurs, dtype: int64

## Query 4 : Sum amounts by week number, where the output displays the NULL bills   

the **'Extrait' sheet in 'Facilis.xlsm' file**, shows total amount of bills, per week number, **even for weeks with no bills**

We aim to get the same output without modifying the table.

* We first create a dataframe for weeks with no bills, and store it into the database as new table

* Then, we make SQL query to get the output

In [42]:
# determine the week numbers in dataframe, which corresponds to the 'factures' table 
df_sql['n_sem'].sort_values().unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 15, 16, 17, 18, 19, 20, 21,
       22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38,
       39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52])

In [29]:
# if False, missing numbers in the range of values from 1 to 52
list_sem = df_sql['n_sem'].sort_values().unique()
list(list_sem) == list(np.arange(1, 53))

False

In [30]:
#df_tmp = df_sql.copy()
#df_tmp.head(2)

In [31]:
#df_tmp[ (df_tmp['n_sem'] > 10) & (df_tmp['n_sem'] < 15) ]

In [32]:
df_sql.shape

(1092, 9)

In [33]:
# initialize list of lists
data = 4*[ ['--'] + [np.nan] + 7*['--']] # 4 : from 11 to 15
                      # 1+1+7=9 : len(df_sql.columns) or df_sql.shape[1]

# Create the pandas DataFrame "temporary dataframe which will correspond to a temporary sql table"
df_tmp = pd.DataFrame(data, columns = df_sql.columns)
df_tmp

,Fournisseurs,n_facture,Montant,Date_de_facture,Date_echeance,n_sem,Mis_en_paie,EMAIL,id_fournisseurs
0,--,NaN,--,--,--,--,--,--,--
1,--,NaN,--,--,--,--,--,--,--
2,--,NaN,--,--,--,--,--,--,--
3,--,NaN,--,--,--,--,--,--,--


In [34]:
df_tmp['Montant']=0
df_tmp = df_tmp.reset_index()
df_tmp

,index,Fournisseurs,n_facture,Montant,Date_de_facture,Date_echeance,n_sem,Mis_en_paie,EMAIL,id_fournisseurs
0,0,--,NaN,0,--,--,--,--,--,--
1,1,--,NaN,0,--,--,--,--,--,--
2,2,--,NaN,0,--,--,--,--,--,--
3,3,--,NaN,0,--,--,--,--,--,--


In [35]:
df_tmp['n_sem'] = df_tmp['index'].apply(lambda x:x+11)

df_tmp['index'] = df_tmp['index'].apply(lambda x:x+1093) # df_sql.shape[0]+1

df_tmp.rename(columns = {'index':'id'}, inplace = True) #id refers to bills


# setting first name as index column
df_tmp.set_index('id', inplace=True)

df_tmp

,Fournisseurs,n_facture,Montant,Date_de_facture,Date_echeance,n_sem,Mis_en_paie,EMAIL,id_fournisseurs
id,,,,,,,,,
1093,--,NaN,0,--,--,11,--,--,--
1094,--,NaN,0,--,--,12,--,--,--
1095,--,NaN,0,--,--,13,--,--,--
1096,--,NaN,0,--,--,14,--,--,--


In [50]:
df_tmp.to_sql('factures_temporary', cnn) # load dataframe 'df' into 'factures_temporary' table

In [44]:
# *******  SQL queries  ********

In [43]:
%%sql

SELECT * FROM factures_temporary

 * sqlite:///data_fournisseurs.db
Done.


id,Fournisseurs,n_facture,Montant,Date_de_facture,Date_echeance,n_sem,Mis_en_paie,EMAIL,id_fournisseurs
1093,--,None,0,--,--,11,--,--,--
1094,--,None,0,--,--,12,--,--,--
1095,--,None,0,--,--,13,--,--,--
1096,--,None,0,--,--,14,--,--,--


In [45]:
# UNION of the two tables

#%%sql

#SELECT * FROM factures

#UNION SELECT * FROM factures_temporary AS new_table

In [39]:
%%sql

WITH total_amount as (SELECT * FROM factures

UNION SELECT * FROM factures_temporary AS new_table )

SELECT n_sem AS Semaine, cast(ROUND(SUM(Montant),0) as int) AS Montants, 

COUNT(n_facture) AS Nbre_Factures

FROM total_amount WHERE n_sem BETWEEN 8 AND 22 GROUP BY n_sem

 * sqlite:///data_fournisseurs.db
Done.


Semaine,Montants,Nbre_Factures
8,543614,15
9,676898,13
10,214533,16
11,0,0
12,0,0
13,0,0
14,0,0
15,346030,21
16,348875,14
17,732764,27


In [41]:
cnn.close()